<a href="https://colab.research.google.com/github/FelipeGBarreto/Projects/blob/master/Rascunho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CASE – ANALISTA DE BUSINESS INTELLIGENCE:

A Fortbrasil tem como um dos seus maiores desafios a entrega de cartões na casa dos nossos clientes, visto que existem área de difícil acesso, outras que apenas o Correios opera, dentre outros. Tendo isso em vista, começamos a diversificar os modais de entrega dos cartões emitidos.

*O case consiste em criar um painel de acompanhamento dos cartões emitidos pela
empresa, e para isso estão sendo enviadas 05 bases:*

1. **“T_CARTÃO”:** Todos os cartões emitidos;
2. **“T_DESBLOQUEIO”:** Indica se o cliente desbloqueou o cartão, considerar apenas o primeiro
desbloqueio de cada ID_CARTÃO;
3. **“T_MODAIS”:** Indica por qual modal o cartão foi enviado e a data;
4. **“T_CANAL_DESBLOQUEIO”:** Registro de por onde foi feito o desbloqueio do cartão, caso não exista nenhum registro de desbloqueio nessa tabela, significa que ele foi feito via “Central de Atendimento”
5. **“T_COMPRAS”:** Registro de compras efetuadas em 2020, assim como seu status.
***


***Pede-se para:***

- Calcular o desbloqueio por safra (mês de emissão) dos cartões de 2020 – acompanhamento
acumulado por 90 dias desde a data de emissão;
- Calcular a ativação (primeira compra) por safra de emissão dos cartões de 2020 -
acompanhamento acumulado por 90 dias desde a data de envio;
- Analisar a efetividade de entrega (podendo ser medida pelo desbloqueio) e sugerir
melhorias;
- Analisar a performance de ativação e sugerir melhorias;

**OBS1:** devem ser considerados cartões válidos apenas os que não começam por ‘9099’ e que tem validade maior que 30 dias.

**OBS2:** use com inteligência as informações ofertadas para gerar um dashboard claro, gerencial e que possa ser usado para tomada de decisões. Gostamos de filtros que facilitem nossas análises.

**OBS3:** segue abaixo o exemplo de gráfico de desbloqueio/ativação acumulada X tempo que pode servir de base.

***

Toda a visualização final deve ser feita em Power BI. Caso precise fazer algum ETL em ferramenta diferente dele, enviar os scripts. É importante que seja enviado junto com o arquivo em Power BI e os scripts (se houverem), uma breve explicação sobre a lógica utilizada e os resultados obtidos.

**Dúvidas:** aline.ricarte@fortbrasil.com.br

# Importando as Bibliotecas Necessárias

In [1]:
import pandas as pd
import numpy as np
import datetime

# Conhecendo os Dados

Conhecendo os dados e veridicando se há valores faltantes, bem como se há valores repetidos, com o fito de um melhor tratamento e precisão dos dados.

In [2]:
df_cartao = pd.read_csv("/content/drive/MyDrive/Emprego/Case Analista 2021 - V2/T_CARTAO.csv", sep=';')
df_desbloqueio = pd.read_excel("/content/drive/MyDrive/Emprego/Case Analista 2021 - V2/T_DESBLOQUEIO.xlsx")
df_modal = pd.read_excel("/content/drive/MyDrive/Emprego/Case Analista 2021 - V2/T_MODAIS.xlsx")
df_canal_desbloq = pd.read_csv("/content/drive/MyDrive/Emprego/Case Analista 2021 - V2/T_CANAL_DESBLOQUEIO.csv", sep=';')
df_compras = pd.read_csv("/content/drive/MyDrive/Emprego/Case Analista 2021 - V2/T_COMPRA.csv", sep=';')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [41]:
t_cartao = df_cartao.copy()
t_desbloqueio = df_desbloqueio.copy()
t_modal = df_modal.copy()
t_canal_desbloq = df_canal_desbloq.copy()
t_compras = df_compras.copy()

In [42]:
def retirar_hr_datas(data,campo):
  data[campo] = data[campo].astype(str)
  separa_data = data[campo].str.split(" ", n = 1, expand = True)
  data[campo] = separa_data[0]
  data[campo] = pd.to_datetime(data[campo])

In [43]:
retirar_hr_datas(t_cartao,"DT_GERACAO_EMBOSSING")
retirar_hr_datas(t_cartao,"DT_VALIDADE")


In [44]:
display(t_compras.shape, t_desbloqueio.shape,t_cartao.shape)

(5823678, 4)

(386373, 2)

(973234, 6)

In [45]:
display(t_compras.sample(2), t_desbloqueio.sample(2),t_cartao.sample(2))

,ID_CONTA,ID_CARTAO,DS_STATUS,DT_COMPRA
965611,840852,4364907,Processada,2020-07-22 19:26:00
3880111,1988848,4603901,Processada,2020-12-05 17:56:00


,ID_CARTAO,DT_DESBLOQUEIO
36998,4149297,2020-03-17
271568,4447669,2020-09-09


,ID_CARTAO,ID_CONTA,NU_SEQ_CARTAO,DT_GERACAO_EMBOSSING,DT_VALIDADE,DS_CARTAO
848053,4120092,1856614,1,2020-02-11,2025-02-13,8596********1945
54360,4280542,1277837,3,2020-04-27,2025-02-13,8596********2223


In [46]:
t_cartao = t_cartao.drop_duplicates()
t_desbloqueio = t_desbloqueio.sort_values("DT_DESBLOQUEIO").drop_duplicates(subset=["ID_CARTAO"], keep='first')
t_compras = t_compras.sort_values(["DT_COMPRA"]).drop_duplicates(subset=["ID_CARTAO"], keep='first')

In [47]:
safra = t_cartao.copy()

In [48]:
desbloqueio_safra = t_cartao.merge(t_desbloqueio, how='left', on='ID_CARTAO')
desbloqueio_safra = desbloqueio_safra[~desbloqueio_safra["DT_DESBLOQUEIO"].isna()]
desbloqueio_safra.head()

,ID_CARTAO,ID_CONTA,NU_SEQ_CARTAO,DT_GERACAO_EMBOSSING,DT_VALIDADE,DS_CARTAO,DT_DESBLOQUEIO
4,4137129,1861372,2,2020-02-19,2025-02-13,8596********6955,2020-03-13
6,4137131,1861373,2,2020-02-19,2025-02-13,8596********0826,2020-02-13
8,4137133,1861374,2,2020-02-19,2025-02-13,8596********0242,2020-02-13
13,4137138,1479019,4,2020-02-13,2025-02-13,8596********2391,2020-02-28
15,4137140,1861377,2,2020-02-19,2025-02-13,8596********9857,2020-03-05


In [49]:
display(desbloqueio_safra.head(2),desbloqueio_safra.shape,desbloqueio_safra.info(), desbloqueio_safra.ID_CARTAO.nunique())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 371543 entries, 4 to 973229
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   ID_CARTAO             371543 non-null  int64         
 1   ID_CONTA              371543 non-null  int64         
 2   NU_SEQ_CARTAO         371543 non-null  int64         
 3   DT_GERACAO_EMBOSSING  371543 non-null  datetime64[ns]
 4   DT_VALIDADE           371543 non-null  datetime64[ns]
 5   DS_CARTAO             371543 non-null  object        
 6   DT_DESBLOQUEIO        371543 non-null  datetime64[ns]
dtypes: datetime64[ns](3), int64(3), object(1)
memory usage: 22.7+ MB


,ID_CARTAO,ID_CONTA,NU_SEQ_CARTAO,DT_GERACAO_EMBOSSING,DT_VALIDADE,DS_CARTAO,DT_DESBLOQUEIO
4,4137129,1861372,2,2020-02-19,2025-02-13,8596********6955,2020-03-13
6,4137131,1861373,2,2020-02-19,2025-02-13,8596********0826,2020-02-13


(371543, 7)

None

371543

In [50]:
ativacao_safra = desbloqueio_safra.merge(t_compras, how='left', on=['ID_CARTAO','ID_CONTA'])
ativacao_safra = ativacao_safra[~ativacao_safra["DT_COMPRA"].isna()]
ativacao_safra.head()

,ID_CARTAO,ID_CONTA,NU_SEQ_CARTAO,DT_GERACAO_EMBOSSING,DT_VALIDADE,DS_CARTAO,DT_DESBLOQUEIO,DS_STATUS,DT_COMPRA
0,4137129,1861372,2,2020-02-19,2025-02-13,8596********6955,2020-03-13,Processada,2020-03-19 11:13:00
1,4137131,1861373,2,2020-02-19,2025-02-13,8596********0826,2020-02-13,Processada,2020-02-13 12:52:00
2,4137133,1861374,2,2020-02-19,2025-02-13,8596********0242,2020-02-13,Processada,2020-02-13 14:14:00
3,4137138,1479019,4,2020-02-13,2025-02-13,8596********2391,2020-02-28,Processada,2020-03-01 19:19:00
4,4137140,1861377,2,2020-02-19,2025-02-13,8596********9857,2020-03-05,Processada,2020-08-07 08:43:00


In [51]:
display(ativacao_safra.head(2),ativacao_safra.shape,ativacao_safra.info(),ativacao_safra.ID_CARTAO.nunique())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 277034 entries, 0 to 371542
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   ID_CARTAO             277034 non-null  int64         
 1   ID_CONTA              277034 non-null  int64         
 2   NU_SEQ_CARTAO         277034 non-null  int64         
 3   DT_GERACAO_EMBOSSING  277034 non-null  datetime64[ns]
 4   DT_VALIDADE           277034 non-null  datetime64[ns]
 5   DS_CARTAO             277034 non-null  object        
 6   DT_DESBLOQUEIO        277034 non-null  datetime64[ns]
 7   DS_STATUS             277034 non-null  object        
 8   DT_COMPRA             277034 non-null  object        
dtypes: datetime64[ns](3), int64(3), object(3)
memory usage: 21.1+ MB


,ID_CARTAO,ID_CONTA,NU_SEQ_CARTAO,DT_GERACAO_EMBOSSING,DT_VALIDADE,DS_CARTAO,DT_DESBLOQUEIO,DS_STATUS,DT_COMPRA
0,4137129,1861372,2,2020-02-19,2025-02-13,8596********6955,2020-03-13,Processada,2020-03-19 11:13:00
1,4137131,1861373,2,2020-02-19,2025-02-13,8596********0826,2020-02-13,Processada,2020-02-13 12:52:00


(277034, 9)

None

277034

# Cálculos

In [52]:
display(safra.head(), desbloqueio_safra.head(), ativacao_safra.head())

,ID_CARTAO,ID_CONTA,NU_SEQ_CARTAO,DT_GERACAO_EMBOSSING,DT_VALIDADE,DS_CARTAO
0,4137125,1861370,2,2020-02-19,2025-02-13,8596********6981
1,4137126,1861371,1,2020-02-19,2025-02-13,8596********1863
2,4137127,1861371,2,2020-02-19,2025-02-13,8596********1096
3,4137128,1861372,1,2020-02-19,2025-02-13,8596********5796
4,4137129,1861372,2,2020-02-19,2025-02-13,8596********6955


,ID_CARTAO,ID_CONTA,NU_SEQ_CARTAO,DT_GERACAO_EMBOSSING,DT_VALIDADE,DS_CARTAO,DT_DESBLOQUEIO
4,4137129,1861372,2,2020-02-19,2025-02-13,8596********6955,2020-03-13
6,4137131,1861373,2,2020-02-19,2025-02-13,8596********0826,2020-02-13
8,4137133,1861374,2,2020-02-19,2025-02-13,8596********0242,2020-02-13
13,4137138,1479019,4,2020-02-13,2025-02-13,8596********2391,2020-02-28
15,4137140,1861377,2,2020-02-19,2025-02-13,8596********9857,2020-03-05


,ID_CARTAO,ID_CONTA,NU_SEQ_CARTAO,DT_GERACAO_EMBOSSING,DT_VALIDADE,DS_CARTAO,DT_DESBLOQUEIO,DS_STATUS,DT_COMPRA
0,4137129,1861372,2,2020-02-19,2025-02-13,8596********6955,2020-03-13,Processada,2020-03-19 11:13:00
1,4137131,1861373,2,2020-02-19,2025-02-13,8596********0826,2020-02-13,Processada,2020-02-13 12:52:00
2,4137133,1861374,2,2020-02-19,2025-02-13,8596********0242,2020-02-13,Processada,2020-02-13 14:14:00
3,4137138,1479019,4,2020-02-13,2025-02-13,8596********2391,2020-02-28,Processada,2020-03-01 19:19:00
4,4137140,1861377,2,2020-02-19,2025-02-13,8596********9857,2020-03-05,Processada,2020-08-07 08:43:00


# Safra

In [53]:
safra["Mes_Geracao"] = safra["DT_GERACAO_EMBOSSING"].dt.month
safra = safra.groupby("Mes_Geracao")[["ID_CARTAO"]].count()
safra.T

Mes_Geracao,1,2,3,4,5,6,7,8,9,10,11,12
ID_CARTAO,170694,66149,65500,47663,46964,67199,72554,74720,67583,81037,99832,113339


# Desbloqueio Safra

In [54]:
desbloqueio_safra["Mes_Geracao"] = desbloqueio_safra["DT_GERACAO_EMBOSSING"].dt.month
desbloqueio_safra = desbloqueio_safra.groupby("Mes_Geracao")[["ID_CARTAO"]].count()
desbloqueio_safra.T

Mes_Geracao,1,2,3,4,5,6,7,8,9,10,11,12
ID_CARTAO,40061,34332,37967,25148,25320,38933,30177,31586,28335,31542,33114,15028


# Ativação Safra

In [55]:
ativacao_safra["Mes_Geracao"] = ativacao_safra["DT_GERACAO_EMBOSSING"].dt.month
ativacao_safra = ativacao_safra.groupby("Mes_Geracao")[["ID_CARTAO"]].count()
ativacao_safra.T

Mes_Geracao,2,3,4,5,6,7,8,9,10,11,12
ID_CARTAO,14289,30460,22589,23600,36494,27881,28743,25082,27535,27895,12466


# Comparativo 

Safra x Desbloqueio x Ativação

In [56]:
comparativo = safra.merge(desbloqueio_safra, how='inner', on='ID_CARTAO')
comparativo

,ID_CARTAO
